In [3]:
import json
import time
import sys
import os
import logging
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
# from semantic_kernel.contents.chat_message_content import ChatMessageContent
# from semantic_kernel.contents.utils.author_role import AuthorRole
# from semantic_kernel.agents import AgentGroupChat
# from semantic_kernel.utils.logging import setup_logging

# from semantic_kernel.contents.chat_history import ChatHistory
# from semantic_kernel.contents.chat_message_content import ChatMessageContent

# parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# if parent_dir not in sys.path:
#     sys.path.insert(0, parent_dir)  # Insert at the beginning for priority
#     print(f"Added {parent_dir} to sys.path")
# else:
#     print(f"{parent_dir} is already in sys.path")

from backend.sk.handler import SemanticKernelHandler
from backend.conversation_store import ConversationStore


load_dotenv(override=True)


ModuleNotFoundError: No module named 'sk'

In [2]:
setup_logging()

logger = logging.getLogger("semantic_kernel")
logger.setLevel(logging.DEBUG)

decorator = logging.getLogger("semantic_kernel.functions.kernel_function_decorator")
decorator.setLevel(logging.INFO)

handler = logging.getLogger("semantic_kernel_handler")
handler.setLevel(logging.DEBUG)

logging.info('Logging set')  # will not print anything

In [3]:
key = DefaultAzureCredential()

db = ConversationStore(
        url=os.getenv("COSMOSDB_ENDPOINT"),
        key=key,
        database_name=os.getenv("COSMOSDB_DATABASE_NAME"),
        container_name=os.getenv("COSMOSDB_CONTAINER_FSI_INS_USER_NAME"),
    )

In [6]:
sk = SemanticKernelHandler(db)

In [5]:
user_data = db.read_user_info('sk_test_ins_4')
# user_data = db.create_user('sk_test_ins_4',{})
# res = await sk.handle_request(user_id='sk_test_ins_1',load_history=True, chat_id='', user_message='',usecase_type='',user_data=[])
# user_data = res['data']
print(user_data)

{'id': 'sk_test_ins_4', '_rid': '4cJyAPnaCHoFAAAAAAAAAA==', '_self': 'dbs/4cJyAA==/colls/4cJyAPnaCHo=/docs/4cJyAPnaCHoFAAAAAAAAAA==/', '_etag': '"1900b37d-0000-0800-0000-67574d830000"', '_attachments': 'attachments/', '_ts': 1733774723}


In [8]:
# await sk.handle_request(user_id='sk_test_ins_1',load_history=False, chat_id='20241016_898', user_message='',usecase_type='',user_data=user_data)
# res = await sk.handle_request(user_id='sk_test_ins_1',load_history=False, chat_id=None, user_message='',usecase_type='',user_data=user_data)
# await sk.handle_request(user_id='sk_test_ins_3',load_history=False, chat_id=None, user_message='Provide me the policies of my client John Doe in a table format',usecase_type='',user_data=user_data)
await sk.handle_request(user_id='sk_test_ins_3',load_history=False, chat_id='20241206_688', user_message='Provide me the policies of my client John Doe in a table format',usecase_type='',user_data=user_data)

[2024-12-06 21:56:35 - semantic_kernel.prompt_template.kernel_prompt_template:68 - DEBUG] Extracting blocks from template: 
                    You are the next speaker selector.

                    - You MUST return ONLY agent name from the list of available agents below.
                    - You MUST return the agent name and nothing else.
                    - Check the history, if any, and decide WHAT agent is the best next speaker
                    - The names are case-sensitive and should not be abbreviated or changed.
                    - YOU MUST OBSERVE AGENT USAGE INSTRUCTIONS.

# AVAILABLE AGENTS

Query: Agent processes the user request by calling external data sources
Summariser: The agent that summarises a final response to the user's original inquiry by collecting responses from the OTHER agents

# CHAT HISTORY

{{$history}}

[2024-12-06 21:56:35 - semantic_kernel.agents.group_chat.agent_chat:112 - INFO] Adding `5` agent chat messages
[2024-12-06 21:56:35 - semantic_

{'status_code': 200,
 'chat_id': '20241206_688',
 'reply': {'role': 'assistant',
  'name': 'Summariser',
  'content': 'Here are the policies for the client John Doe:\n\n| Policy No | Basic Plan Product | Product Type               | Policy Status | Effective Date         | Expiry Date            | Overseas Medical Expenses | Follow up Treatment in Malaysia | Alternative Medicine | Overseas Hospital Income | Child Care Benefit | Compassionate Visit | Emergency Medical Evacuation | Medical Repatriation | Repatriation of Mortal Remains |\n|-----------|--------------------|----------------------------|---------------|------------------------|------------------------|---------------------------|-------------------------------|----------------------|--------------------------|-------------------|---------------------|-----------------------------|----------------------|-------------------------------|\n| 0         | P230ZII            | Ztravel International      | A             | 2024-06-18

### SK Agent Group Chat - Single Agent / multi skill test - no history single shot

In [ ]:
handler = SemanticKernelHandler(db)

userId = 'sk_test_ins_1'
user_data = {'chat_histories': {}}

chat_history = ChatHistory()

team_query_single = 'Provide me the policies of my client John Doe in a table format'

agent_group_chat = handler.create_agent_group_chat(chat_history)

await agent_group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=team_query_single))

async for item in agent_group_chat.invoke():
    print("---------------- >>>>>>  -------------------")
    print(f"{item.name}: {item.content}")
    continue

response = list(reversed([item async for item in agent_group_chat.get_chat_messages()]))

print(f"SK AgentGroupChat reponder response = {response}")

[2024-12-06 02:25:25 - semantic_kernel.prompt_template.kernel_prompt_template:68 - DEBUG] Extracting blocks from template: 
                    You are the next speaker selector.

                    - You MUST return ONLY agent name from the list of available agents below.
                    - You MUST return the agent name and nothing else.
                    - Check the history, if any, and decide WHAT agent is the best next speaker
                    - The names are case-sensitive and should not be abbreviated or changed.
                    - YOU MUST OBSERVE AGENT USAGE INSTRUCTIONS.

# AVAILABLE AGENTS

Query: Agent processes the user request by calling external data sources
Summariser: The agent that summarises a final response to the user's original inquiry by collecting responses from the OTHER agents

# CHAT HISTORY

{{$history}}
                    
[2024-12-06 02:25:25 - semantic_kernel.agents.group_chat.agent_chat:112 - INFO] Adding `1` agent chat messages
[2024-12-06 

---------------- >>>>>>  -------------------
Query: Here are the policies for the client John Doe:

| Policy No | Basic Plan Product | Product Type               | Policy Status | Effective Date         | Expiry Date            | Overseas Medical Expenses | Follow up Treatment in Malaysia | Alternative Medicine | Overseas Hospital Income | Child Care Benefit | Compassionate Visit | Emergency Medical Evacuation | Medical Repatriation | Repatriation of Mortal Remains |
|-----------|--------------------|----------------------------|---------------|------------------------|------------------------|---------------------------|-------------------------------|----------------------|--------------------------|-------------------|---------------------|-----------------------------|----------------------|-------------------------------|
| 0         | P230ZII            | Ztravel International      | A             | 2024-06-18 00:00:00.000| 2024-07-13 00:00:00.000| 500000                    | 500

[2024-12-06 02:25:31 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:194 - INFO] OpenAI usage: CompletionUsage(completion_tokens=3, prompt_tokens=1483, total_tokens=1486, completion_tokens_details=None, prompt_tokens_details=None)
[2024-12-06 02:25:31 - semantic_kernel.functions.kernel_function:29 - INFO] Function selection succeeded.
[2024-12-06 02:25:31 - semantic_kernel.functions.kernel_function:39 - DEBUG] Function result: Summariser
[2024-12-06 02:25:31 - semantic_kernel.functions.kernel_function:53 - INFO] Function completed. Duration: 0.218935s
[2024-12-06 02:25:31 - semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:84 - INFO] Kernel Function Selection Strategy next method completed: , selection, result: [ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-AbHNzwc1LIwZCyPIdVS1AkqvyHPG7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Summariser', refusal=None, role='assis

---------------- >>>>>>  -------------------
Summariser: Here are the policies for the client John Doe:

| Policy No | Basic Plan Product | Product Type               | Policy Status | Effective Date         | Expiry Date            | Overseas Medical Expenses | Follow up Treatment in Malaysia | Alternative Medicine | Overseas Hospital Income | Child Care Benefit | Compassionate Visit | Emergency Medical Evacuation | Medical Repatriation | Repatriation of Mortal Remains |
|-----------|--------------------|----------------------------|---------------|------------------------|------------------------|---------------------------|-------------------------------|----------------------|--------------------------|-------------------|---------------------|-----------------------------|----------------------|-------------------------------|
| 0         | P230ZII            | Ztravel International      | A             | 2024-06-18             | 2024-07-13             | 500,000                   

In [10]:

print(response[-1].name)
print(response[-1].role)
print(response[-1].content)


Summariser
AuthorRole.ASSISTANT
Here are the policies for the client John Doe:

| Policy No | Basic Plan Product | Product Type               | Policy Status | Effective Date         | Expiry Date            | Overseas Medical Expenses | Follow up Treatment in Malaysia | Alternative Medicine | Overseas Hospital Income | Child Care Benefit | Compassionate Visit | Emergency Medical Evacuation | Medical Repatriation | Repatriation of Mortal Remains |
|-----------|--------------------|----------------------------|---------------|------------------------|------------------------|---------------------------|-------------------------------|----------------------|--------------------------|-------------------|---------------------|-----------------------------|----------------------|-------------------------------|
| 0         | P230ZII            | Ztravel International      | A             | 2024-06-18             | 2024-07-13             | 500,000                   | 50,000                 

In [ ]:
def extract_sk_responses(response):
    """
    Extracts the assistant's response from a list of ChatMessageContent objects.

    Args:
        chat_messages (list): A list of ChatMessageContent objects.

    Returns:
        dict: A dictionary with keys 'role', 'name', and 'content' containing the assistant's response.
    """
    # filtered_messages = filter(lambda m: m.content and m.role in (AuthorRole.ASSISTANT, AuthorRole.USER), response)
    results = [ 
        {**d, 'name': d.get('name', '')} 
        for d in (obj.to_dict() for obj in response) 
    ]
    
    return results



In [6]:
extract_sk_responses(response)

[{'role': 'user',
  'content': 'Provide me the policies of my client John Doe in a table format',
  'name': ''},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_dCgKnMAjSy91QJR0zvHwyges',
    'type': 'function',
    'function': {'name': 'crm-load_from_crm_by_client_fullname',
     'arguments': '{"full_name":"John Doe"}'}}],
  'name': 'Query'},
 {'role': 'tool',
  'content': '{"id": "987654321", "clientID": "987654321", "firstName": "John", "lastName": "Doe", "fullName": "John Doe", "dateOfBirth": "1980-01-01", "nationality": "Malaysia", "contactDetails": {"email": "john.doe@example.com", "phone": "123-456-7890"}, "address": {"street": "123 Main St", "city": "New York", "state": "NY", "postalCode": "12345", "country": "USA"}, "policies": [{"PolicyNo": "0", "BasicPlanProduct": "P230ZII", "ProductType": "Ztravel International", "PolicyStatus": "A", "EffectiveDate": "2024-06-18 00:00:00.000", "ExpiryDate": "2024-07-13 00:00:00.000", "Overseas Medical Expenses": "500000", "Follow up Tr